<img src='https://github.com/Ikomia-dev/notebooks/blob/main/examples/img/banner_ikomia.png?raw=true'>




# Easy Object Tracking with DeepSORT

**DeepSORT** (Simple Online and Realtime Tracking with a Deep Association Metric) is an extension of the original SORT (Simple Real-time Tracker) algorithm, which is considered an elegant and widely used framework for object tracking. It incorporates a deep learning methodology to address real-world tracking challenges such as occlusions and different viewpoints.

![illustration Deepsort](https://learnopencv.com/wp-content/uploads/2022/06/01-sprint-race.gif)

## Setup

You need to install Ikomia Python API with pip


In [ ]:
!pip install ikomia

---

*Note: The script is not compatible with Google Colab as they have disabled cv2.imshow()*

---

## Download video and cut example video

In [1]:

import requests
import cv2

url = "https://www.pexels.com/download/video/12116094/?fps=29.97&h=720&w=1280"

# Define headers to mimic a browser request
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"
}

response = requests.get(url, headers=headers, stream=True)
with open("video.mp4", "wb") as f:
    for chunk in response.iter_content(chunk_size=1024):
        f.write(chunk)

# Replace with the path to your downloaded video
video_path = "video.mp4"

# Open the video
cap = cv2.VideoCapture(video_path)

# Check if the video has opened successfully
if not cap.isOpened():
    print("Error: Could not open video.")
    exit()

# Get video properties
fps = cap.get(cv2.CAP_PROP_FPS)
frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
duration = frame_count / fps
cut_frame = int(frame_count / 4)  # Frame to cut the video at

# Define the codec and create VideoWriter object
fourcc = cv2.VideoWriter_fourcc(*'mp4v') 
out = cv2.VideoWriter('short_video.mp4', fourcc, fps, (int(cap.get(3)), int(cap.get(4))))

# Read and write frames until the cut point
frame_num = 0
while True:
    ret, frame = cap.read()
    if not ret or frame_num == cut_frame:
        break
    out.write(frame)
    frame_num += 1

# Release everything
cap.release()
out.release()

## Run YOLOv7 and DeepSORT on your video

In [ ]:
from ikomia.dataprocess.workflow import Workflow
from ikomia.utils.displayIO import display
import cv2


# Replace 'your_video_path.mp4' with the actual video file path
input_video_path = 'short_video.mp4'
output_video_path = 'deepsort_output_video.avi'

# Init your workflow
wf = Workflow()

# Add object detection algorithm
detector = wf.add_task(name="infer_yolo_v7", auto_connect=True)

# Add ByteTrack tracking algorithm
tracking = wf.add_task(name="infer_deepsort", auto_connect=True)

tracking.set_parameters({
    "categories": "all",
    "conf_thres": "0.5",
})

# Open the video file
stream = cv2.VideoCapture(input_video_path)
if not stream.isOpened():
    print("Error: Could not open video.")
    exit()

# Get video properties for the output
frame_width = int(stream.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(stream.get(cv2.CAP_PROP_FRAME_HEIGHT))
frame_rate = stream.get(cv2.CAP_PROP_FPS)

# Define the codec and create VideoWriter object
# The 'XVID' codec is widely supported and provides good quality
fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter(output_video_path, fourcc, frame_rate, (frame_width, frame_height))

while True:
    # Read image from stream
    ret, frame = stream.read()

    # Test if the video has ended or there is an error
    if not ret:
        print("Info: End of video or error.")
        break

    # Run the workflow on current frame
    wf.run_on(array=frame)

    # Get results
    image_out = tracking.get_output(0)
    obj_detect_out = tracking.get_output(1)

    # Convert the result to BGR color space for displaying
    img_out = image_out.get_image_with_graphics(obj_detect_out)
    img_res = cv2.cvtColor(img_out, cv2.COLOR_RGB2BGR)

    # Save the resulting frame
    out.write(img_out)

    # Display
    display(img_res, title="DeepSORT", viewer="opencv")

    # Press 'q' to quit the video processing
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# After the loop release everything
stream.release()
out.release()
cv2.destroyAllWindows()